In [3]:
import os
import copy
import glob
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import math
from tqdm import tqdm

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn import MSELoss
from torchvision import transforms
from torch.optim import lr_scheduler
from torch.nn.modules.activation import ReLU, Sigmoid
from torch.nn import Conv2d, modules
from torch.nn import Sequential

import torch.backends.cudnn as cudnn

In [2]:
!pip install opencv-python

In [4]:
# utils functions
def img_read(fPath):
    '''
    read the image given path "fPath"
    '''
    img = cv2.imread(fPath, -1) # single channel image
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def getLR(hr_img, scale):
    '''
    bibubic downsmaple to get the corresponding low-resolution image by scaling factor "scale"
    '''
    h_orig, w_orig = hr_img.shape
    h, w = int(h_orig/scale), int(w_orig/scale)
    lr_img = cv2.resize(hr_img, (w, h), interpolation=cv2.INTER_NEAREST) # 30*30
    lr_img = cv2.resize(lr_img, (w_orig, h_orig), interpolation=cv2.INTER_CUBIC) # 120*120
    return lr_img

def computePSNR(img1, img2):
    '''
    compute PSNR(Peak Signal to Noise Ratio) to calculate accuracy
    img1 and img2 have range [0, 1], and both are gray level images
    '''
    if not img1.shape == img2.shape:
        print("Input images must have the same dimensions.")
    mse = torch.mean((img1-img2)**2)
    if mse == 0: # img1 and img2 are same images
        return float('inf')
    return 10.0 * torch.log10(1.0/mse)

In [5]:
class SRCNN(nn.Module):
    def __init__(self, num_channels=3):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=num_channels, 
                               out_channels=64, 
                               kernel_size=9, 
                               padding=9 // 2)
        self.conv2 = nn.Conv2d(in_channels=64, 
                               out_channels=32, 
                               kernel_size=5, 
                               padding=5 // 2)
        self.conv3 = nn.Conv2d(in_channels=32, 
                               out_channels=num_channels, 
                               kernel_size=5, 
                               padding=5 // 2)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

In [6]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [7]:
class SRData(Dataset):
    def __init__(self, imgFolder, scale, transform=None):
        self.imgFolder = imgFolder
        self.scale = scale
        self.transform = transform
        self.images = sorted(glob.glob(self.imgFolder + '/*.png')) # all image patch path
    
    def __getitem__(self, index):
        hr_img = img_read(self.images[index])
        #h = hr_img.shape[0]
        #w = hr_img.shape[1]
        '''
        pre_trans = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((int(h/self.scale),int(w/self.scale)), interpolation=Image.NEAREST),
            transforms.Resize((h,w), interpolation=Image.BICUBIC)
        ])
        '''
        #lr_img = pre_trans(hr_img)
        lr_img = getLR(hr_img, self.scale)
        if self.transform:
            hr_img = self.transform(hr_img)
            lr_img = self.transform(lr_img)
        return lr_img, hr_img

    def __len__(self):
        return len(self.images)

In [8]:
'''
hyper parameters
'''
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda":
    print("cuda available")
    cudnn.benchmark = True

BATCH_SIZE = 16
NUM_WORKERS = 0
LR = 1e-4
EPOCHS = 400

IMG_FOLDER = "dataset"
print(len(os.listdir(IMG_FOLDER)))
SCALE = 3
verbose = 1

cuda available
70946


In [9]:
'''
prepare data
'''
# convert input to tensor
train_transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.0), (1.0)) # do nothing
])

val_transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.0), (1.0)) # do nothing
])

data = SRData(imgFolder=IMG_FOLDER, 
              scale=SCALE, 
              transform=train_transformer)

train_indices = torch.arange(49666)
val_indices = torch.arange(49666, 63859)

train_data = torch.utils.data.Subset(data, train_indices)
val_data = torch.utils.data.Subset(data, val_indices)
# data size 7 : 2 : 1
print("train set length: {}".format(int(len(train_data))))
print("val set length: {}".format(int(len(val_data))))
# load data
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              num_workers=NUM_WORKERS,
                              pin_memory=True,
                              shuffle=True)
val_dataloader = DataLoader(dataset=val_data,
                            batch_size=1)

train set length: 49666
val set length: 14193


In [10]:
'''
training the SRCNN model
'''
from torch.utils.tensorboard import SummaryWriter

total_train_step = 0    # total training step
total_val_step = 0      # total validation step

# build model
srcnn = SRCNN(num_channels=1).to(DEVICE)
# loss function
loss_fn = nn.MSELoss()
#loss_fn = loss_fn.to(DEVICE)
# optimizer
optimizer = torch.optim.Adam([
    {'params': srcnn.conv1.parameters()},
    {'params': srcnn.conv2.parameters()},
    {'params': srcnn.conv3.parameters(),'lr': LR * 0.1}], lr=LR)
# learning rate scheduler
#scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5, verbose=True)
# visualize (tensorboard)
writer = SummaryWriter("logs")

best_weights = copy.deepcopy(srcnn.state_dict())
best_epoch = 0
best_psnr = 0.0

torch.cuda.empty_cache()

count = 0


for i in range(EPOCHS):
    
    # training
    srcnn.train()
    epoch_losses = AverageMeter()
    
    with tqdm(total=(len(train_data)), ncols=100) as t1:
        t1.set_description('epoch train: {}/{}'.format(i+1, EPOCHS))
        
        for data in train_dataloader:
            
            # get data, transpose to device
            lr_imgs, hr_imgs = data
            lr_imgs = lr_imgs.to(DEVICE)
            hr_imgs = hr_imgs.to(DEVICE)
            
            # predict
            outputs = srcnn(lr_imgs)
            
            # calculate loss
            loss = loss_fn(outputs, hr_imgs)
            epoch_losses.update(loss.item(), len(lr_imgs))
            
            # optimizer
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # print loss
            t1.set_postfix(loss='{:.6f}'.format(epoch_losses.avg))

            # show on tensorboard
            total_train_step += 1            
            if total_train_step%1000 == 0:
            #    print("train step: {}，Loss: {}".format(total_train_step, loss.item()))
                writer.add_scalar("train_loss", epoch_losses.avg, total_train_step)
            
            # update tqdm
            t1.update(len(lr_imgs))
    
        # scheduler
        #scheduler.step()

    # validation
    if (i+1) % verbose == 0:
    
        srcnn.eval()
        epoch_psnr = AverageMeter()

        for data in val_dataloader:
            
            # get data, transpose to device
            lr_imgs, hr_imgs = data
            lr_imgs = lr_imgs.to(DEVICE)
            hr_imgs = hr_imgs.to(DEVICE)
            
            # predict (no_grad) is important, not use to update model
            # cut compute graphe to reduce needed memory of device and 
            # accelerate the computation
            with torch.no_grad():
                outputs = srcnn(lr_imgs).clamp(0.0, 1.0)
            
            # calculate psnr
            psnr =  (outputs, hr_imgs)
            # update total psnr
            epoch_psnr.update(psnr, len(lr_imgs))
        
        # print psnr
        print('eval PSNR: {:.2f}'.format(epoch_psnr.avg))
        
        # show in tensorboard
        total_val_step += verbose
        writer.add_scalar("val_psnr", epoch_psnr.avg, total_val_step) 

        # save best weights
        if epoch_psnr.avg > best_psnr:
            best_epoch = i
            best_psnr = epoch_psnr.avg
            best_weights = copy.deepcopy(srcnn.state_dict())
    

# close tensorboard
writer.close()    

# save best model
print('best epoch: {}, psnr: {:.2f}'.format(best_epoch, best_psnr))
torch.save(best_weights, "saved_models/srcnn_{}.pth".format(best_epoch))
print("best model saved")

epoch train: 1/400: 100%|█████████████████████| 49666/49666 [03:21<00:00, 246.01it/s, loss=0.001411]


eval PSNR: 40.84


epoch train: 2/400: 100%|█████████████████████| 49666/49666 [01:53<00:00, 438.25it/s, loss=0.000197]


eval PSNR: 43.80


epoch train: 3/400: 100%|█████████████████████| 49666/49666 [02:00<00:00, 413.75it/s, loss=0.000175]


...

eval PSNR: 46.47


epoch train: 399/400: 100%|███████████████████| 49666/49666 [01:13<00:00, 673.63it/s, loss=0.000061]


eval PSNR: 46.46


epoch train: 400/400: 100%|███████████████████| 49666/49666 [01:13<00:00, 672.41it/s, loss=0.000061]


eval PSNR: 46.51
best epoch: 373, psnr: 46.55
best model saved
